In [1]:
import pygame
import os
import time
import random

pygame.init()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


(6, 0)

In [2]:
WIDTH,HEIGHT=600,600
WIN=pygame.display.set_mode((WIDTH,HEIGHT))
pygame.display.set_caption("SPACE INVADER")

In [3]:
RED_SPACE_SHIP=pygame.transform.scale(pygame.image.load(os.path.join("assets","pixel_ship_red_small.png")),(50,50))
GREEN_SPACE_SHIP=pygame.transform.scale(pygame.image.load(os.path.join("assets","pixel_ship_green_small.png")),(50,50))
BLUE_SPACE_SHIP=pygame.transform.scale(pygame.image.load(os.path.join("assets","pixel_ship_blue_small.png")),(50,50))
YELLOW_SPACE_SHIP=pygame.transform.scale(pygame.image.load(os.path.join("assets","pixel_ship_yellow.png")),(65,65))

In [4]:
RED_LASER=pygame.image.load(os.path.join("assets","pixel_laser_red.png"))
GREEN_LASER=pygame.image.load(os.path.join("assets","pixel_laser_green.png"))
BLUE_LASER=pygame.image.load(os.path.join("assets","pixel_laser_blue.png"))
YELLOW_LASER=pygame.image.load(os.path.join("assets","pixel_laser_yellow.png"))

In [5]:
BG=pygame.transform.scale(pygame.image.load(os.path.join("assets","background-black.png")),(WIDTH,HEIGHT))

In [6]:
class Laser:
    def __init__(self,x,y,img):
        self.x=x
        self.y=y
        self.img=img
        self.mask=pygame.mask.from_surface(self.img)
        
    def draw(self,window):
        window.blit(self.img,(self.x-18,self.y))
        
    def move(self,vel):
        self.y+=vel
        
    def off_screen(self,height):
        return not(self.y<=height and self.y>=0)
    
    def collision(self,obj):
        return collide(obj,self) #to check whether the obj is colliding to myself or not
    

In [7]:
class Ship:
    COOLDOWN=30 #its half a sec cz FPS is 60
    def __init__(self,x,y,health=100):
        self.x=x
        self.y=y
        self.health=health
        self.ship_img=None
        self.laser_img=None
        self.lasers=[]
        self.cool_down_counter=0
    
    def draw(self,window):
        window.blit(self.ship_img,(self.x,self.y))
        for laser in self.lasers:
            laser.draw(window)
            
    def move_lasers(self,vel,obj):
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            elif laser.collision(obj):
                obj.health-=10
                self.lasers.remove(laser)
    
    def cooldown(self):
        if self.cool_down_counter>=self.COOLDOWN:
            self.cool_down_counter=0
        elif self.cool_down_counter>0:
            self.cool_down_counter+=1
    
    def get_width(self):
        return self.ship_img.get_width()
    
    def get_height(self):
        return self.ship_img.get_height()
        
    def shoot(self):
        if self.cool_down_counter==0:
            laser=Laser(self.x,self.y,self.laser_img)
            self.lasers.append(laser)
            self.cool_down_counter=1

In [8]:
class Player(Ship):
    def __init__(self,x,y,health=100):
        super().__init__(x,y,health) #this initializes all the initializations in the ship class __init__ fn
        self.ship_img=YELLOW_SPACE_SHIP
        self.laser_img=YELLOW_LASER
        self.mask=pygame.mask.from_surface(self.ship_img) #mask tells us where pixels are/arent in this ship_img so that when we hv collision we knw if we actually hit a pixel or not
        self.max_health=health
        
    def move_lasers(self,vel,objs):
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            else:
                for obj in objs:
                    if laser.collision(obj):
                        objs.remove(obj)
                        self.lasers.remove(laser)
                        
    def draw(self,window):
        super().draw(window)
        self.healthbar(window)
                        
    def healthbar(self,window):
        pygame.draw.rect(window,(255,0,0),(self.x,self.y+self.ship_img.get_height()+10,self.ship_img.get_width(),10))
        pygame.draw.rect(window,(0,255,0),(self.x,self.y+self.ship_img.get_height()+10,self.ship_img.get_width()*(self.health/self.max_health),10))

In [9]:
class Enemy(Ship):
    
    COLOR_MAP={
                "red":(RED_SPACE_SHIP,RED_LASER),
                "green":(GREEN_SPACE_SHIP,GREEN_LASER),
                "blue":(BLUE_SPACE_SHIP,BLUE_LASER),
              }
    
    def __init__(self,x,y,color,health=100):
        
        super().__init__(x,y,health)
        self.ship_img,self.laser_img=self.COLOR_MAP[color]
        self.mask=pygame.mask.from_surface(self.ship_img)
        
    def move(self,vel):
        self.y+=vel

In [10]:
def collide(obj1,obj2):     #collision of pixels of the obj not the edges of the pic'c width/height
    offset_x=obj2.x-obj1.x  #dist of x cordinate of top left of obj1 from obj2
    offset_y=obj2.y-obj1.y  #dist of y cordinate of top left of obj1 from obj2      
    return obj1.mask.overlap(obj2.mask,(offset_x,offset_y))!=None  #given mask of both the obj are overlapping or not
                                                                   #returns true if it returns a tuple like (x,y) , else returns false if it returns None

In [11]:
def main():
    run=True
    FPS=60
    level=0
    lives=5
    main_font=pygame.font.SysFont("comicsans",30)
    lost_font=pygame.font.SysFont("comicsans",70)
    
    enemies=[]
    wave_length=5
    enemy_vel=1
    
    laser_vel=5
    
    player_vel=5 #velocity of the movement of the player
    
    player=Player(270,510)
    
    
    clock=pygame.time.Clock()
    
    lost=False
    lost_count=0
    
    def redraw_window():
        WIN.blit(BG,(0,0)) #initialize starting point of the bg which is top left 
        # draw text
        lives_lable=main_font.render(f"Lives: {lives}",1,(255,255,255)) #render("",1,(R,G,B))--->always use this format to blit the text to the game in the desired color using RGB color code
        level_lable=main_font.render(f"Level: {level}",1,(255,255,255))
        
        WIN.blit(lives_lable,(10,10))
        WIN.blit(level_lable,(WIDTH-level_lable.get_width()-10,10))
       
        for enemy in enemies:
            enemy.draw(WIN)
        
        player.draw(WIN)
        
        if lost:
            lost_lable=lost_font.render("YOU LOST..!!",1,(255,255,255))
            WIN.blit(lost_lable,(WIDTH/2-lost_lable.get_width()/2,HEIGHT/2+lost_lable.get_height()/2))
                
        pygame.display.update()
    
    while run:
        clock.tick(FPS)
        
        redraw_window()
        
        if lives<=0 or player.health<=0:
            lost=True
            lost_count+=1
            
        if lost:
            if lost_count>FPS*3: #a timer for 3 sec before executing further
                run=False
            else:
                continue
        
        if len(enemies)==0:
            level+=1
            wave_length+=5
            for i in range(wave_length):
                enemy=Enemy(random.randrange(50,WIDTH-100),random.randrange(-1500,-100),random.choice(["red","blue","green"]))
                enemies.append(enemy)
                
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                run=False
        
        keys=pygame.key.get_pressed()
        if keys[pygame.K_a] and player.x-player_vel>0: #left
            player.x-=player_vel
        
        if keys[pygame.K_d] and player.x+player_vel+player.get_width()<WIDTH: #right
            player.x+=player_vel
            
        if keys[pygame.K_w] and player.y-player_vel>0: #up
            player.y-=player_vel
            
        if keys[pygame.K_s] and player.y+player_vel+player.get_height()+15<HEIGHT: #down
            player.y+=player_vel
            
        if keys[pygame.K_SPACE]:
            player.shoot()
            
        for enemy in enemies[:]:
            enemy.move(enemy_vel)
            enemy.move_lasers(laser_vel,player)
            
            if random.randrange(0,2*60)==1:
                enemy.shoot()
            
            if enemy.y+enemy.get_height()>HEIGHT:
                lives-=1
                enemies.remove(enemy)
            elif collide(enemy,player):
                player.health-=10
                enemies.remove(enemy)
                
        player.move_lasers(-laser_vel,enemies)
    
def main_menu():
        pygame.font.init()
        title_font=pygame.font.SysFont("comicsans",50)
        run=True
        while run:
            WIN.blit(BG,(0,0))
            title_lable=title_font.render("Press the mouse to begin...",1,(255,255,255))
            WIN.blit(title_lable,(WIDTH/2-title_lable.get_width()/2,HEIGHT/2+title_lable.get_height()/2))
            pygame.display.update()
            for event in pygame.event.get():
                if event.type==pygame.QUIT:
                    run=False
                if event.type==pygame.MOUSEBUTTONDOWN:
                    main()
        pygame.quit()
    
        
        

In [12]:
main_menu()